In [ ]:
import pandas as pd
import textwrap
import hashlib

# Display full content for large text columns
pd.set_option('display.max_colwidth', None)  
# Display all rows (adjust if dataset is extremely large)
pd.set_option('display.max_rows', 200)  # Set to a higher number if needed
pd.set_option('display.max_columns', None)  # Show all columns

# Path to the questions dataset
FILE_PATH = 'C:\\Users\\kuzne\\Documents\\Python_repo\\2025_01_dissertation\\2025_dissertation\\data\\MedQA_no-opt_dataset_with_reasoning.xlsx'  # Replace with your file path


In [ ]:
# Load the Excel file
df = pd.read_excel(FILE_PATH)

# Create a list of psychiatric keywords to search for
psychiatric_keywords = [
    'ptsd',
    'major depressive disorder', 
    'anxiety', 
    'schizophrenia', 
    'bipolar', 
    'manic',
    'psychosis', 
    'mood disorder', 
    'adhd', 
    'autism',
    'eating disorder', 
    'bulimia', 
    , 
    'acute stress disorder', 
    'ocd', 
    'panic attack', 
    'personality disorder'
]


# Function to check if any psychiatric keyword is in the text
def contains_psychiatric_term(text):
    if isinstance(text, str):
        return any(keyword.lower() in text.lower() for keyword in psychiatric_keywords)
    return False

In [3]:
# Count questions related to each psychiatric condition
psychiatric_counts = {}


for keyword in psychiatric_keywords:
    answer_count = df['Reasonings'].str.contains(keyword, case=False).sum() if 'Reasonings' in df.columns else 0
    
    total_count = answer_count
    if total_count > 0:
        psychiatric_counts[keyword] = total_count

In [4]:
# Display results
print("Number of questions per psychiatric condition:")
for condition, count in sorted(psychiatric_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{condition}: {count}")

# Total psychiatric questions
total_psychiatric = sum(psychiatric_counts.values())
print(f"\nTotal number of psychiatric-related questions: {total_psychiatric}")
print(f"Total number of questions in dataset: {len(df)}")
print(f"Percentage of psychiatric questions: {(total_psychiatric/len(df)*100):.2f}%")

Number of questions per psychiatric condition:
anxiety: 252
bipolar: 89
schizophrenia: 73
major depressive disorder: 57
personality disorder: 38
manic: 31
adhd: 26
psychosis: 22
panic attack: 21
autism: 18
ocd: 18
mood disorder: 15
eating disorder: 14
ptsd: 13
bulimia: 8
acute stress disorder: 1

Total number of psychiatric-related questions: 696
Total number of questions in dataset: 10178
Percentage of psychiatric questions: 6.84%


In [5]:
def create_question_id(question, answer):
    """
    Create a unique ID based on complete question and answer content.
    Returns full MD5 hash to minimize collision risk.
    """
    combined = (question + answer).encode('utf-8')
    return hashlib.md5(combined).hexdigest()  # Return full 32-character hash

In [6]:
def filter_question(df, condition, output_file='filtered_questions_with_ids.xlsx'):
    mask = df['Reasonings'].str.contains(condition, case=False, na=False)
    # Boolean mask to filter rows with the condition
    condition_df = df[mask].copy()

    if len(condition_df) == 0:
        print(f"No questions found containing '{condition}'")
        return None

    # Create unique IDs for each question
    condition_df['question_id'] = condition_df.apply(
        lambda row: create_question_id(row['Modified Questions'], row['Reasonings']), 
        axis=1
    )

    # Save to Excel with condition name in filename
    output_filename = f"{condition}_{output_file}"
    condition_df.to_excel(output_filename, index=False)
    
    print(f"Found {len(condition_df)} questions about {condition}")
    print(f"Saved to {output_filename}")
    
    return condition_df

In [7]:
data = filter_question(df, 'ptsd')

Found 13 questions about ptsd
Saved to ptsd_filtered_questions_with_ids.xlsx


In [19]:

def review_condition_questions(condition, 
                               input_file='filtered_questions_with_ids.xlsx', 
                               output_file='reviewed_questions.xlsx',  width=80):
    
    # Try to load previous progress
    # If we already reviewed some questions, we want to keep track of them.
    # This is why we load OUTPUT file first, to check whether it exists.
    excel_path = f'{condition}_{output_file}'
    try:
        previous_df = pd.read_excel(excel_path)
        reviewed_ids = set(previous_df['question_id'])
        print(f"Found {len(reviewed_ids)} previously reviewed questions")
        
        
        # If we don't have any previous results, create an empty DataFrame.
    except FileNotFoundError:
        reviewed_ids = set()
        previous_df = pd.DataFrame()  # Create empty DataFrame for new results
        print("Starting new review session")

    # Load filtered file with questions about the condition.
    # File is created by the filter_question function.
    input_filename = f"{condition}_{input_file}"
    condition_df = pd.read_excel(input_filename)
    
    # Filter out already reviewed questions
    remaining_df = condition_df[~condition_df['question_id'].isin(reviewed_ids)]
    remaining_df['is_appropriate'] = None  # Add a column to mark valid questions

    print(f"Found {len(remaining_df)} questions about {condition}")
    print("For each question, enter:")
    print("1 - Appropriate")
    print("0 - Not appropriate")
    print("q - Quit review")

    for idx, row in remaining_df.iterrows():
        # Wrap question and answer text
        # https://docs.python.org/3/library/textwrap.html
        # Wraps the single paragraph in text, and returns a single 
        # string containing the wrapped paragraph
        question = textwrap.fill(row['Modified Questions'], width=width, 
                               initial_indent="Question: ", subsequent_indent="          ")
        answer = textwrap.fill(row['Reasonings'], width=width,
                             initial_indent="Answer: ", subsequent_indent="        ")
        
        print(f"\n{question}\n")
        print(f"{answer}\n")

        # Part for manual evaluation of the question quality. 
        while True:
            response = input("Is this question appropriate? (1/0/q): ").lower()
            if response in ['1', '0', 'q']:
                break
            print("Invalid input. Please enter 1, 0, or q.")
        
        if response == 'q':
            break
        
        remaining_df.at[idx, 'is_appropriate'] = int(response)


    # Save reviewed questions
    reviewed_df = remaining_df[remaining_df['is_appropriate'].notna()]
    # Separate Dataframe with only appropriate questions
    appropriate_df = reviewed_df[reviewed_df['is_appropriate'] == 1]
    
    if not reviewed_df.empty:
        if not previous_df.empty:
            combined_df = pd.concat([previous_df, reviewed_df], ignore_index=True)

        else:
            combined_df = reviewed_df

        # Save to Excel (OUTPUT path)
        combined_df.to_excel(excel_path, index=False)

        # Append to text file
        mode = 'w' if previous_df.empty else 'a'
        with open(f'{condition}_review_results.txt', mode, encoding='utf-8') as f:
            f.write(f"\nSession: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*50 + "\n\n")

            for _, row in appropriate_df.iterrows():
                wrapped_question = textwrap.fill(row['Modified Questions'], 
                                               width=width,
                                               initial_indent="Question: ",
                                               subsequent_indent="         ")
                wrapped_answer = textwrap.fill(row['Reasonings'], 
                                             width=width,
                                             initial_indent="Answer: ",
                                             subsequent_indent="       ")
                f.write(f"{wrapped_question}\n\n")
                f.write(f"{wrapped_answer}\n\n")
                f.write("-"*width + "\n\n")
            
            # Write session summary
            f.write("\nSESSION SUMMARY\n")
            f.write("="*50 + "\n")
            f.write(f"Questions reviewed this session: {len(reviewed_df)}\n")
            f.write(f"New appropriate questions: {len(appropriate_df)}\n")
            f.write(f"Total reviewed questions: {len(combined_df)}\n\n")


        print(f"\nSaved {len(reviewed_df)} new questions to {excel_path}")
        print(f"Appropriate questions appended to {condition}_review_results.txt")

    

In [20]:
# Example usage:
condition_to_review = "ptsd"  # Change this to the condition you want to review
review_condition_questions(condition_to_review,  width=70)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_6320\3096481408.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_df['is_appropriate'] = None  # Add a column to mark valid questions


Found 6 previously reviewed questions
Found 7 questions about ptsd
For each question, enter:
1 - Appropriate
0 - Not appropriate
q - Quit review

Question: A 32-year-old man comes to the Veterans Affairs hospital
          because of difficulty sleeping for the past 9 weeks. He is a
          soldier who returned from a deployment in Afghanistan 12
          weeks ago. Fifteen weeks ago, his unit was ambushed in a
          deserted street, and a fellow soldier was killed. He wakes
          up frequently during the night from vivid dreams of this
          incident. He blames himself for being unable to save his
          friend. He also has trouble falling asleep and gets up
          earlier than desired. During this period, he has started to
          avoid walking in deserted streets. Vital signs are within
          normal limits. Physical examination shows no abnormalities.
          He refuses cognitive behavioral therapy and is started on
          sertraline. Five weeks later


Question: A 27-year-old Caucasian female presents to her physician for
          episodes of urinary incontinence that began shortly after a
          breakup with her boyfriend. She claimed to be
          psychologically devastated when she found him sleeping with
          her brother and has had trouble caring for herself ever
          since. The patient states that the episodes came on suddenly
          and occur randomly. The patient denies any burning or pain
          upon urination. Upon obtaining further history, the patient
          also states that she has "stress spells" in which her vision
          becomes blurry or has blind spots. The patient also
          complains of frequent headaches. These symptoms have
          persisted for the past few years and she attributes them to
          arguments with her boyfriend. Embarrassed, the patient even
          admits to episodes of fecal incontinence which she also
          blames on her boyfriend's perpetual verbal a

In [3]:
def search_terms_in_excel(file_path, search_terms):
    """
    Search for specific terms across all sheets in an Excel workbook.
    
    Parameters:
    -----------
    file_path : str
        Path to the Excel file
    search_terms : list
        List of terms to search for
    
    Returns:
    --------
    pandas.DataFrame
        Combined dataframe of all matches with sheet names included
    """
    # Load the workbook
    excel_data = pd.ExcelFile(file_path)
    
    # Store results
    results = []
    
    # Search through each sheet
    for sheet in excel_data.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet)
        
        # Filter rows that contain search terms in any column
        matched_rows = df[df.apply(lambda row: any(term.lower() in str(row).lower() 
                                                  for term in search_terms), axis=1)]
        
        # Add sheet name if matches found
        if not matched_rows.empty:
            matched_rows['Sheet Name'] = sheet
            results.append(matched_rows)
    
    # Combine and return results
    if results:
        final_results = pd.concat(results, ignore_index=True)
        return final_results
    else:
        return pd.DataFrame() 

In [4]:
search_terms = ['major depressive disorder']
search_terms_in_excel(FILE_PATH, search_terms)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Modified Questions,Reasonings,Sheet Name
0,"Please refer to the summary above to answer this question\nAn ECG is most likely to show which of the following findings in this patient?""\n""Patient Information\nAge: 64 years\nGender: F, self-identified\nEthnicity: unspecified\nSite of Care: emergency department\nHistory\nReason for Visit/Chief Concern: “My chest hurts, especially when I take a deep breath.”\nHistory of Present Illness:\n2-hour history of chest pain\npain described as “sharp”\npain rated 6/10 at rest and 10/10 when taking a deep breath\nPast Medical History:\nrheumatoid arthritis\nmajor depressive disorder\nMedications:\nmethotrexate, folic acid, fluoxetine\nAllergies:\npenicillin\nPsychosocial History:\ndoes not smoke\ndrinks one glass of bourbon every night\nPhysical Examination\nTemp Pulse Resp BP O2 Sat Ht Wt BMI\n36.","The patient's symptoms of sharp chest pain, particularly when taking a deep breath, are consistent with myocardial infarction (heart attack). This is supported by the patient's history of risk factors such as age, gender, and smoking. Additionally, the patient's physical examination reveals tachycardia and hypertension, which are also consistent with myocardial infarction. Therefore, an ECG would likely show diffuse, concave ST-segment elevations, indicating injury to multiple areas of the heart muscle. Thus, the answer is Diffuse, concave ST-segment elevations.",Train
1,"A 51-year-old male presents to his primary care provider for a normal check-up. He reports that he “hasn’t felt like himself” recently. He describes feeling down for the past 8 months since his mother passed away. He has had trouble sleeping and has unintentionally lost 15 pounds. He feels guilty about his mother’s death but cannot articulate why. His performance at work has declined and he has stopped running, an activity he used to enjoy. He has not thought about hurting himself or others. Of note, he also complains of numbness in his feet and fingers and inability to maintain an erection. His past medical history is notable for diabetes. He is on metformin. His temperature is 98.6°F (37°C), blood pressure is 125/65 mmHg, pulse is 90/min, and respirations are 16/min. On exam, he is alert and oriented with intact memory and normal speech. He appears tired with a somewhat flattened affect. Corresponding to the signs and symptoms, the patient presents with and the investigations performed already, what immediate measures of management are to be initiated, prioritising their urgency, and keeping in mind the probable diagnosis for the same?","The patient's presentation of decreased mood, anhedonia, insomnia, weight loss, guilt, and decreased interest in activities he once enjoyed, along with physical symptoms such as numbness in the feet and fingers, suggests a diagnosis of major depressive disorder (MDD). MDD can often occur in individuals following a significant loss, such as the death of a loved one. The patient's history of diabetes increases the likelihood of neuropathy contributing to the numbness. The patient's vital signs, particularly his blood pressure and tachycardia, suggest sympathetic nervous system activation, which is consistent with depression. Given the patient's symptoms and history, the most appropriate immediate measure of management would be to initiate antidepressant therapy, specifically norepinephrine and serotonin reuptake inhibitors (SNRIs), which target both norepinephrine and serotonin neurotransmitters. SNRIs have been shown to be effective in treating MDD and have a more rapid onset of action compared to other classes of antidepressants. Thus, the answer is norepinephrine and serotonin reuptake inhibitors.",Train
2,A 43-year-old woman is found in the hospital to have a plasma sodium concentration of 126 mg/dL. She was hospitalized after she expressed suicidal ideations and was started on a medication for major depressive disorder. Her past medical history is significant for diabetes for which she i

In [5]:
search_terms = ['PTSD', 'post-traumatic stress disorder']
search_terms_in_excel(FILE_PATH, search_terms)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Modified Questions,Reasonings,Sheet Name
0,"A 65-year-old man presents to the emergency department for sudden weakness. The patient states that he was at home enjoying his morning coffee when his symptoms began. He says that his left arm suddenly felt very odd and weak thus prompting him to come to the ED. The patient has a past medical history of diabetes, COPD, hypertension, anxiety, alcohol abuse, and PTSD. He recently fell off a horse while horseback riding but claims to not have experienced any significant injuries. He typically drinks 5-7 drinks per day and his last drink was yesterday afternoon. His current medications include insulin, metformin, atorvastatin, lisinopril, albuterol, and fluoxetine. His temperature is 99.5°F (37.5°C), blood pressure is 177/118 mmHg, pulse is 120/min, respirations are 18/min, and oxygen saturation is 93% on room air. On physical exam, you note an elderly man who is mildly confused. Cardiopulmonary exam demonstrates bilateral expiratory wheezes and a systolic murmur along the right upper sternal border that radiates to the carotids. Neurological exam reveals cranial nerves II-XII as grossly intact with finger-nose exam mildly abnormal on the left and heel-shin exam within normal limits. The patient has 5/5 strength in his right arm and 3/5 strength in his left arm. The patient struggles to manipulate objects such as a pen with his left hand. The patient is given a dose of diazepam and started on IV fluids. What is the most likely diagnosis in this patient?","The patient's presentation of sudden weakness in his left arm, along with mild confusion, bilateral expiratory wheezes, and a systolic murmur, suggests a possible diagnosis of lipohyyalinosis. This condition is characterized by the accumulation of fat in the soft tissues of the neck, which can cause compression of the cranial nerves, leading to symptoms such as weakness or paralysis of the facial, tongue, and limb muscles. The patient's history of alcohol abuse and recent fall from a horse may also contribute to the development of this condition. Additionally, the patient's mild confusion and difficulty manipulating objects with his left hand are consistent with a cranial nerve deficit. Thus, the answer is Lipohyalinosis.",Train
1,"A 42-year-old man presents to his primary care provider complaining of insomnia. He describes 3 months of frequent nighttime awakenings and nightmares. Per chart review, he is a combat veteran and was on a military tour in Afghanistan 4 months ago when a car bomb exploded, injuring him and killing his friend; however, when the physician asks about this, the patient states that he “does not talk about that” and changes the subject. He reports anxiety, irritability and feeling detached from his friends and family, which he believes is harming his relationships. Physical exam reveals an overweight, anxious appearing man with normal vital signs and an exaggerated startle response. What is the pharmacological action of the medications that might have helped prevent this patient's current disorder?","Posttraumatic stress disorder (PTSD) is a psychiatric condition that can develop after a person experiences or witnesses a traumatic event. Symptoms include recurrent nightmares, flashbacks, avoidance of triggers, and changes in mood and behavior. In this case, the patient's symptoms of insomnia, nightmares, anxiety, irritability, and detachment are consistent with PTSD. Propranolol is a beta blocker that has been shown to reduce the physical and psychological symptoms associated with PTSD by blocking the effects of adrenaline, a hormone released during times of stress. By reducing the body's response to stress, propranolol may help alleviate the patient's symptoms and prevent them from worsening. Thus, the answer is Propanolol.",Train
2,"An 18-year-old female presents to general medical clinic with the report of a rape on her college campus. The patient was visiting a local fraternity, and after having a few drinks,

In [6]:
search_terms = ['schizophrenia']
search_terms_in_excel(FILE_PATH, search_terms)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Modified Questions,Reasonings,Sheet Name
0,"A 30-year-old man presents with restlessness and an inability to sit or lie down for the past 2 days. Past medical history is significant for schizophrenia, diagnosed 3 weeks ago and managed medically. Vital signs are a blood pressure of 140/90 mm Hg and a pulse of 96/min. On physical examination, the patient is fidgety and anxious but well-oriented. What according to you is the probable diagnosis?","(Please provide your response here)\n\nAkathisia is a common side effect of antipsychotic medications, which are used to treat conditions such as schizophrenia. It is characterized by a feeling of inner restlessness or agitation that can manifest as an inability to sit or lie down, as well as a desire to move around constantly. The patient's recent diagnosis of schizophrenia and initiation of medication therapy suggest that akathisia may be the underlying cause of his symptoms. Additionally, the patient's vital signs, particularly his elevated blood pressure and heart rate, support this diagnosis. The absence of other symptoms such as fever, chest pain, or shortness of breath also point towards akathisia as the most likely diagnosis. Thus, the answer is Akathisia.",Train
1,"A 23-year-old patient with a past medical history of anxiety and appropriately treated schizophrenia presents to the emergency department for a first time seizure. The patient was at home eating dinner when he began moving abnormally and did not respond to his mother, prompting her to bring him in. His symptoms persisted in the emergency department and were successfully treated with diazepam. The patient is discharged and scheduled for a follow up appointment with neurology the next day for treatment. The patient returns to his neurologist 1 month later for a checkup. Physical exam is notable for carpopedal spasm when his blood pressure is being taken. Cranial nerves II-XII are grossly intact and his gait is stable. What are the most likely explanation of this patient's current presentation","The patient has a history of taking antipsychotics which can induce the P450 system, leading to increased metabolism of other drugs such as benzodiazepines like diazepam. This can lead to reduced efficacy of these medications over time, causing a return of symptoms such as seizures. Additionally, the patient's carpopedal spasms suggest hypocalcemia, a known side effect of P450 induction. Thus, the answer is P450 induction.",Train
2,A 29-year-old woman is brought to the physician by her father because of a change in her behavior over the past 8 months. The father says that his daughter has become increasingly withdrawn; she has not answered any phone calls or visited her family and friends. The patient says that she has to stay at home because a foreign intelligence service is monitoring her. She thinks that they are using a magnetic field to read her mind. Mental status exam shows disjointed and perseverative thinking. She is anxious and has a flat affect. What is the diagnosis?,"(Please fill in the blank)\n\nThe patient's symptoms of withdrawal, paranoid delusions, disjointed and perseverative thinking, anxiety, and flat affect are all consistent with schizophrenia. Her belief that a foreign intelligence service is monitoring her and using a magnetic field to read her mind is an example of a delusion, which is a common symptom of schizophrenia. Additionally, her disorganized thinking and speech patterns, as well as her lack of motivation and emotional expression, are also indicative of the disorder. These symptoms suggest that the patient is experiencing a psychotic episode, which is a classic feature of schizophrenia. Thus, the answer is schizophrenia.",Train
3,"A 23-year-old woman is brought to the emergency department by her boyfriend because of a 4-month history of feeling sad. Her boyfriend says that, during this period, she has slept and eaten very little and has been unable to focus at work. She says that she feels “empt

In [7]:
search_terms = ['borderline personality disorder']
search_terms_in_excel(FILE_PATH, search_terms)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet


,Modified Questions,Reasonings,Sheet Name
0,"A 53-year-old woman with type 2 diabetes mellitus is admitted for evaluation of recurrent episodes of nausea, tremors, and excessive sweating. She works as a nurse and reports self-measured blood glucose levels below 50 mg/dL on several occasions. Her family history is positive for borderline personality disorder. The only medication listed in her history is metformin. After metformin administration, what is the next management step","Metformin is associated with hypoglycemia, which can cause symptoms such as nausea, tremors, and excessive sweating. However, the patient's self-reported blood glucose levels are consistently low, suggesting that she may be experiencing factitious hypoglycemia or intentionally lowering her blood sugar levels to induce these symptoms. This behavior is consistent with borderline personality disorder, which is known to be associated with manipulative behavior and factitious disorders. Therefore, it is important to ask the patient if she is taking any medications other than metformin, as this could help to confirm or rule out the diagnosis of factitious hypoglycemia. Thus, the answer is ""Ask the patient if she is taking any medications other than metformin.""",Train
1,"A 29-year-old woman is hospitalized due to depression and suicidal ideation. She has a 5-year history of chaotic relationships that last only a few short weeks or months. Each relationship has left her feeling abandoned, empty, and extremely upset. During these periods, the patient confesses to shopping and making big purchases on impulse. She says she gets bored easily and moves on to the next adventure. The patient denies any changes in appetite, energy level, or concentration. On examination, multiple linear lacerations of varying phases of healing were noted on her forearms and trunk. Following consultation, she praises physicians to be ‘the best people on the planet’, but when the nurse came in to take her blood, she furiously stated that ‘all nurses are incompetent and cruel’. What is the diagnosis?","The patient's symptoms suggest a pattern of unstable interpersonal relationships, emotional dysregulation, and impulsivity, which are characteristic features of borderline personality disorder (BPD). Her tendency to idealize and then suddenly devalue others, as seen in her shifting attitudes towards physicians and nurses, is also consistent with BPD. Additionally, her self-destructive behavior, such as cutting herself, and her impulsive behaviors, like excessive spending, further support this diagnosis. Finally, her difficulty regulating her emotions and experiencing intense feelings of abandonment and emptiness also align with the diagnostic criteria for BPD. Thus, the answer is Borderline personality disorder.",Train
2,"A 26-year-old woman presents to her gynecologist with complaints of pain with her menses and during intercourse. She also complains of chest pain that occurs whenever she has her menstrual period. The patient has a past medical history of bipolar disorder and borderline personality disorder. Her current medications include lithium and haloperidol. Review of systems is notable only for pain when she has a bowel movement relieved by defecation. Her temperature is 98.2°F (36.8°C), blood pressure is 114/74 mmHg, pulse is 70/min, respirations are 14/min, and oxygen saturation is 98% on room air. Pelvic exam is notable for a tender adnexal mass. The patient's uterus is soft, boggy, and tender. Describe the diagnosis of this patient","The patient's symptoms of painful periods, painful intercourse, and chest pain occurring during her menstrual cycle suggest endometriosis. The presence of an adnexal mass and a tender uterus supports this diagnosis. Additionally, the patient's history of bipolar disorder and borderline personality disorder may indicate a higher likelihood of developing chronic pelvic pain. To confirm the diagnosis and assess the extent of the disease, a laparosco

In [8]:
search_terms = ['bipolar disorder']
search_terms_in_excel(FILE_PATH, search_terms)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Modified Questions,Reasonings,Sheet Name
0,"A 45-year-old woman comes to the physician because of a 2-week history of fatigue and excessive thirst. During this period, she has not been able to sleep through the night because of the frequent urge to urinate. She also urinates more than usual during the day. She drinks 4–5 liters of water and 1–2 beers daily. She has autosomal dominant polycystic kidney disease, hypertension treated with lisinopril, and bipolar disorder. Therapy with valproic acid was begun after a manic episode 3 months ago. Vital signs are within normal limits. Irregular flank masses are palpated bilaterally. The remainder of the examination shows no abnormalities. Laboratory studies show:\nSerum\nNa+ 152 mEq/L\nK+ 4.1 mEq/L\nCl− 100 mEq/L\nHCO3− 25 mEq/L\nCreatinine 1.8 mg/dL\nOsmolality 312 mOsmol/kg\nGlucose 98 mg/dL\nUrine osmolality 190 mOsmol/kg\nThe urine osmolality does not change after 3 hours despite no fluid intake or after administration of desmopressin. What is the next best step for managing this patient?","The patient's symptoms of fatigue, excessive thirst, and frequent urination suggest central diabetes insipidus (CDI). CDI is caused by a lack of antidiuretic hormone (ADH), which leads to an increase in urine production and dilution of the urine. The patient's urine output, osmolality, and response to desmopressin support the diagnosis of CDI. Hydrochlorothiazide therapy would help manage the patient's symptoms by increasing ADH secretion and reducing urine production. Thus, the answer is Hydrochlorothiazide therapy.",Train
1,"A 58-year-old man with a past medical history of diabetes, hypertension, and hyperlipidemia was brought into the emergency department by his wife after she observed him go without sleep for several days and recently open and max out several credit cards. She also reports that he has quit his bartending job and has been excessively talkative and easily annoyed for the last several weeks. The patient has no previous psychiatric history. Routine medical examination, investigations, and toxicology rule out a medical cause or substance abuse. Lab results are consistent with chronically impaired renal function. What is the single best treatment for this patient?","The patient's presentation of decreased need for sleep, increased talkativeness, and easy irritability, along with recent onset of impulsive behavior, suggests a diagnosis of mania. This is further supported by the absence of a previous psychiatric history and the presence of chronic medical conditions such as diabetes, hypertension, and hyperlipidemia. Mania is often associated with bipolar disorder, but it can also occur as a side effect of certain medications or medical conditions. In this case, valproic acid, an anticonvulsant commonly used to treat seizures, migraines, and bipolar disorder, would be an appropriate choice for treating the patient's manic symptoms. Valproic acid has a proven efficacy in controlling manic episodes and its use is well established in clinical practice guidelines. Thus, the answer is Valproic acid.",Train
2,"A 40-year-old woman presents to her primary care physician with a 5-month history of worsening bladder discomfort. Her discomfort is relieved by voiding. She voids 10–15 times per day and wakes up 2–3 times per night to void. She has not had any involuntary loss of urine. She has tried cutting down on fluids and taking NSAIDs to reduce the discomfort with minimal relief. Her past medical history is significant for bipolar disorder. She is sexually active with her husband but reports that intercourse has recently become painful. Current medications include lithium. Her temperature is 37°C (98.6°F), pulse is 65/min, and blood pressure is 110/80 mm Hg. Examination shows tenderness to palpation of her suprapubic region. Urinalysis shows:\nColor clear\npH 6.7\nSpecific gravity 1.010\nProtein 1+\nGlucose negative\nKetones negative\nBlood negative\nNitrite negative\nLeukocyte esterase n

In [11]:
search_terms = ['personality disorder']
search_terms_in_excel(FILE_PATH, search_terms)

C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_rows['Sheet Name'] = sheet
C:\Users\kuzne\AppData\Local\Temp\ipykernel_8056\4258461946.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Modified Questions,Reasonings,Sheet Name
0,"A 69-year-old male presents to the emergency room with back pain. He has a history of personality disorder and metastatic prostate cancer and was not a candidate for surgical resection. He began chemotherapy but discontinued due to unremitting nausea. He denies any bowel or bladder incontinence. He has never had pain like this before and is demanding morphine. The nurse administers IV morphine and he feels more comfortable. Vital signs are stable. On physical examination you note tenderness to palpation along the lower spine, weakness in the bilateral lower extremities, left greater than right. Neurological examination is also notable for hyporeflexia in the knee and ankle jerks bilaterally. You conduct a rectal examination, which reveals saddle anesthesia. What is the diagnosis and the management of the case","Cauda equina syndrome is a surgical emergency that requires prompt recognition and treatment. It is characterized by symptoms such as new onset of bladder or bowel incontinence, saddle anesthesia, bilateral sciatica, and decreased anal tone. In this case, the patient's history of metastatic prostate cancer increases the likelihood of a spinal metastasis causing cauda equina compression. The patient's presentation of bilateral lower extremity weakness, hyporeflexia, and saddle anesthesia supports this diagnosis. While waiting for MRI confirmation, steroids should be initiated to reduce inflammation and swelling in the affected area, potentially alleviating some of the patient's symptoms and preventing further neurological damage. Thus, the answer is <Given Answer>.",Train
1,"A 27-year-old man is brought to a psychiatrist by his mother who is concerned that he has become increasingly distant. When asked, he says that he is no longer going out because he is afraid of going outside by himself. He says that ever since he was a teenager, he was uncomfortable in large crowds and on public transportation. He now works from home and rarely leaves his house except on mandatory business. What is etiopathogenesis of above presenting signs and symptoms?","The patient's symptoms of avoidance behavior, fear of going outside by himself, and preference for working from home suggest that he may have developed a dependent personality disorder. This disorder is characterized by a pervasive pattern of dependence, passivity, and need for others to take responsibility for major areas of life. People with this disorder often have difficulty making decisions and initiating actions, and may feel uncomfortable or helpless when faced with new situations or challenges. They may also have a tendency to avoid social interactions and prefer to stay at home, as seen in this case. Additionally, the patient's history of feeling uncomfortable in large crowds and on public transportation suggests that he may have developed some level of social anxiety, which can also contribute to his avoidance behavior. Thus, the answer is Dependent.",Train
2,"A 12-year-old boy is brought in by his parents as they are concerned about his behavior. He is constantly arguing with his parents and blatantly defying their every request. In school, he is known for being intentionally tardy and for defying his teachers. Upon further questioning of the patient you learn about some of his recent behaviors such as beginning smoking to bother his parents. You also notice that he is disrespectful towards you. You then learn that he has also gotten into weekly fights with his peers at school with no apparent cause. In addition, last week he was seen hitting one of the local neighborhood cats with a baseball bat trying to kill it. The patient lives at home with his two parents in a pre-World War II house that was recently renovated. Physical exam is unremarkable.\n\nLaboratory values are as follows:\n\nNa: 140 mmol/L\nK: 4.5 mmol/L\nCl: 100 mmol/L\nGlucose: 80 mg/dL\nCa: 10 mmol/L\nMg: 3 mEq/L\nCr: 0.8 mg/dL\nBUN: 10 mg/dL\nSerum lead: < .01 